In [1]:
from numpy import sin, cos, pi, exp
from scipy.signal import welch
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed, FloatSlider, IntSlider, HBox, VBox, interactive_output, Layout
import ipywidgets as widget

%matplotlib ipympl

In [2]:
def testOutput(n_start, n_stop, domain):
    print(n_start, '\n', 
          n_stop, '\n',
          domain)
    win_stop.min = n_start+1


domainSelection = widget.RadioButtons(
    options=['Time Trace', 'Frequency Spectrum'],
    value='Time Trace',
    description='Display: ',
    disabled=False,
    continuous_update=False
)
win_start = widget.BoundedIntText(    
    value=4000,
    min=0,
    max=100000,
    step=1,
    description='Signal segment start (sample number):',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=Layout(width='95%'),
    continuous_update=False
)
win_stop = widget.BoundedIntText(    
    value=8000,
    min=0,
    max=10000,
    step=1,
    description='Signal segment stop (sample number):',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=Layout(width='95%'),
    continuous_update=False
)
layout = HBox(
    [VBox([win_start, win_stop]), 
     domainSelection]
)
userInput = {
    'n_start': win_start,
    'n_stop': win_stop,
    'domain': domainSelection
}
out = interactive_output(testOutput, userInput)
display(layout, out)

Output()

In [3]:


def sliderPanelSetup(set_details, n_of_sets=1, slider_type='float'):
    panel_col = []
    sliders = {}
    for i in range(n_of_sets):
        panel_row = []
        for item in set_details:
            mathtext = item['description']
            mathtext = mathtext.strip('$')
            if n_of_sets > 1:
                if mathtext.find(" ") == -1:
                    mathtext = '$' + mathtext + '_' + str(i+1) + '$' 
                else:
                    mathtext = '$' + mathtext.replace(" ", '_'+str(i+1)+'\ ', 1) + '$'
            else:
                mathtext = '$' + mathtext + '$'
            #mathtext = r'{}'.format(mathtext)

            panel_row.append(FloatSlider(value=item['value'], 
                                         min=item['min'],
                                         max = item['max'], 
                                         step = item['step'], 
                                         description=mathtext, 
                                         layout=Layout(width='95%')))
            
            sliders[item['keyword']+str(i+1)] = panel_row[-1]
        panel_col.append(HBox(panel_row, layout = Layout(width='100%')))
    layout = VBox(panel_col, layout = Layout(width='90%'))
    return sliders, layout

class y_of_x_plot:
    def __init__(self, ax, t, A_max, N=1, t_unit='s'):
        res  = len(t)
        self.N = N
        t_nd = np.outer(t, np.ones(self.N))
        x_t = np.zeros((res, self.N))          

        self.ax = ax
        self.lines = self.ax.plot(t_nd, x_t)
        
        # avgrensning av akser, rutenett, merkede punkt på aksene, tittel, aksenavn
        self.ax.axis([t[0], t[-1], -A_max, A_max])
        self.ax.grid(True)
        #self.ax.set_xticks(np.linspace(t[0],t[-1],11))
        self.ax.set_xlabel("Tid (" + t_unit + ")")
        
    def update(self, new_lines_x, new_lines_y):
        assert self.N == len(new_lines_x) == len(new_lines_y), "Error: Parameter lenght different from number of sines."
        for i in range(self.N):
            self.lines[i].set_ydata(new_lines_y[i])
            self.lines[i].set_xdata(new_lines_x[i])
            
    def setLabels(self, names):
        self.ax.legend(self.lines, names, loc='upper right')
        
    def setStyles(self, styles):
        for i in range(min(len(styles), len(self.lines))):
            try:
                self.lines[i].set_color(styles[i]['color'])
            except:
                pass
            
            try:
                self.lines[i].set_linestyle(styles[i]['linestyle'])
            except:
                pass


In [9]:
# Demo 4:
# Sum av sinusbølger med vektoraddisjon        
class signalAnalyzer:
    def __init__(self, x_n, f_s, fig_num=1):
        self.x_n = x_n
        self.f_s = f_s
        self.t = np.linspace(0, len(self.x_n)/self.f_s, len(self.x_n), endpoint=False)
        
        
        # Set up canvas
        plt.close(fig_num)
        self.fig = plt.figure(fig_num, figsize=(9, 6))
        
        
        # Set up plot showing signal selection
        self.ax1 = plt.subplot(4, 1, 1)
        self.ax1.set_title(r"Full Signal Plot")
        self.ax1.plot(self.t, self.x_n,  color='tab:blue')
        self.ax1.grid(True)
        self.ax1.set_xlabel('Time t (seconds)')
        self.highlight, = self.ax1.plot(self.t, self.x_n, color='tab:orange')
        self.ax1.axis(xmin=self.t[0], xmax=self.t[-1])
        
        
        
        # Set up signal segment inspection plot
        self.ax2 = plt.subplot(4, 1, (2,4))
        
        self.selectionCurve, = self.ax2.plot(self.t, self.x_n, color='tab:blue')
        self.ax2.grid(True)
        
        
        # Adjust figure layout
        self.fig.tight_layout(pad=0.1, w_pad=1.0, h_pad=2.0)
        
        # Set up UI panel
        domainSelection = widget.RadioButtons(
            options=['Time Trace', 'Frequency Spectrum'],
            value='Time Trace',
            description='Display: ',
            disabled=False,
            continuous_update=False
        )
        win_start = widget.BoundedIntText(    
            value=4000,
            min=0,
            max=100000,
            step=1,
            description='Signal segment start (sample number):',
            disabled=False,
            style = {'description_width': 'initial'},
            layout=Layout(width='95%'),
            continuous_update=False
        )
        win_stop = widget.BoundedIntText(    
            value=8000,
            min=0,
            max=10000,
            step=1,
            description='Signal segment stop (sample number):',
            disabled=False,
            style = {'description_width': 'initial'},
            layout=Layout(width='95%'),
            continuous_update=False
        )
        self.layout = HBox(
            [VBox([win_start, win_stop]), 
             domainSelection]
        )
        self.userInput = {
            'n_start': win_start,
            'n_stop': win_stop,
            'domain': domainSelection
        }
        out = interactive_output(self.update, self.userInput)
        display(self.layout, out)
        # Run demo
        #out = interactive_output(self.update, self.sliders)
        #display(self.layout, out)
        
    def update(self, n_start, n_stop, domain):
        self.userInput['n_stop'].min=n_start+1
        self.highlight.set_ydata(self.x_n[n_start:n_stop])
        self.highlight.set_xdata(self.t[n_start:n_stop])
        if domain=='Time Trace':
            self.ax2.set_xlabel('Time t (seconds)')
            self.ax2.set_ylabel('Value x(t)')
            self.selectionCurve.set_ydata(self.x_n[n_start:n_stop])
            self.selectionCurve.set_xdata(self.t[n_start:n_stop])
            self.ax2.set_xlabel("Time t (seconds)")
            self.ax2.set_ylabel("Value x(t)")
            self.ax2.set_title("Time plot of selected signal segment")
            self.ax2.relim()
            self.ax2.autoscale_view()
            self.ax2.axis([self.t[n_start], self.t[n_stop], min(self.x_n)*1.1, max(self.x_n)*1.1])
            
        elif domain=='Frequency Spectrum':
            M = n_stop-n_start
            f, Sxx_sub = welch(self.x_n[n_start:n_stop], self.f_s, 'hamming', int(M/4), int(M/8), int(M/2))
            
            self.ax2.set_xlabel("Frequency f (Hz)")
            self.ax2.set_ylabel("Power Pxx(f) (dB)")
            self.ax2.set_title("Frequency content of selected signal segment")
            self.selectionCurve.set_ydata(10*np.log10(Sxx_sub))
            self.selectionCurve.set_xdata(f)
            self.ax2.relim()
            self.ax2.autoscale_view()
            self.ax2.axis(xmin=0, xmax=self.f_s/2)
            
        else:
            pass
        self.fig.tight_layout(pad=0.1, w_pad=1.0, h_pad=2.0)


signalAnalyzer(np.linspace(0, 1, 10000, endpoint=False), 8000)              

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

In [7]:
np.maximum([1, 2, 3])

ValueError: invalid number of arguments